In [ ]:
%matplotlib notebook 
# Import modules
import os
import math
import numpy as np
import matplotlib.pyplot
import re
import pandas as pd
from scipy.stats import norm
from pyne import serpent
from pyne import nucname
import time
import multiprocessing as mp

In [ ]:
def read_keff(index):
    res_file = dir+prefix+str(index)+'_res.m'
    res = serpent.parse_res(res_file, write_py=False)
    return res['IMP_KEFF'][:,0]
    
def read_comp(index):
    dep_file = dir+prefix+str(index)+'_dep.m'
    dep = serpent.parse_dep(dep_file, write_py=False,make_mats=False)
    return dep['MAT_fuel_MDENS']
    
dir = '../sampler/'
prefix = 'tap_'
fnum_start = 0   # first file index
fnum_end = 999  # last file index
iso_list = ['U235', 'U236','U238', 'Pu239', 'Pu240', 'Xe135', 'I135']


ncpu = mp.cpu_count()
# res_d = serpent.parse_res(dir+prefix+str(0)+'_res.m', write_py=False)
dep_d = serpent.parse_dep(dir+prefix+str(0)+'_dep.m', write_py=False,make_mats=False)
days = dep_d['DAYS']
vol_fuel = dep_d['MAT_fuel_VOLUME'][0]
iso_names = dep_d['NAMES'][0].split()
#print(vol_fuel)
#print(iso_names)

In [ ]:
start_time = time.time()
with mp.Pool(processes = ncpu) as p:
    k_ds_list = p.map(read_keff, [idx for idx in range(0, fnum_end-fnum_start+1)])
#d = read_comp(0)
#print(vol_fuel*d[iso_names.index('U235'),:])
k_ds_array = np.array(k_ds_list)
elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [ ]:
#np.savetxt('keff_1000_samples.gz', k_ds_array)
k_ds_array = np.loadtxt('keff_1000_samples.gz')

In [ ]:
start_time = time.time()
with mp.Pool(processes = ncpu) as p:
    depletion_ds = p.map(read_comp, [idx for idx in range(0, fnum_end-fnum_start+1)])
depletion = np.array(depletion_ds)
#print(vol_fuel*depletion[iso_names.index('U235'),:])
elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [ ]:
#np.savez_compressed('depleted_mass_1000_samples.npz', a=depletion)
with np.load('depleted_mass_1000_samples.npz', allow_pickle=True) as data:
    depletion = data['a']

In [ ]:
k_eff = []
std_k = []
rel_err = []
nsamp, nt = k_ds_array.shape

for t in range(nt):
    k_eff.append(np.median(k_ds_array[:,t]))
    std_k.append(1e+5*np.std(k_ds_array[:,t]))
    rel_err.append(100*np.std(k_ds_array[:,t])/np.median(k_ds_array[:,t]))

print(len(k_eff))            
#print(std_k)
print(r"Keff at BOC %5.5f +- %5.1f pcm; Std/mean %5.3f %%" %(k_eff[0],std_k[0], rel_err[0]))
print(r"Keff at EOC %5.5f +- %5.1f pcm; Std/mean %5.3f %%" %(k_eff[-1],std_k[-1], rel_err[-1]))

In [ ]:
mass = {}
mass_std = {}
mass_rel = {}
nsampl, isos_number, nt = np.shape(depletion)

print(nsampl, isos_number, nt)
for iso in iso_list:
    iso_idx = iso_names.index(iso)
    mass[iso] = []
    mass_std[iso] = []
    mass_rel[iso] = []
    for t in range(nt):
        mass[iso].append(1e-3*vol_fuel*np.median(depletion[:,iso_idx,t]))
        mass_std[iso].append(1e-3*vol_fuel*np.std(depletion[:,iso_idx,t]))
        mass_rel[iso].append(100* np.std(depletion[:,iso_idx,t])/np.median(depletion[:,iso_idx,t]))

#print(mass)            
#print(mass_std)
#print(mass_rel)
print("U235 mass at BOC %7.3f +- %5.3f kg; Std/mean %5.3f %%" %(mass['U235'][1],mass_std['U235'][1], mass_rel['U235'][1]) )
print("U235 mass at EOC %7.3f +- %5.3f kg; Std/mean %5.3f %%" %(mass['U235'][-1],mass_std['U235'][-1], mass_rel['U235'][-1]) )
print("\nU238 mass at BOC %7.3f +- %5.3f kg; Std/mean %5.3f %%" %(mass['Pu239'][1],mass_std['Pu239'][1], mass_rel['Pu239'][1]) )
print("U238 mass at EOC %7.3f +- %5.3f kg; Std/mean %5.3f %%" %(mass['Pu239'][-1],mass_std['Pu239'][-1], mass_rel['Pu239'][-1]) )

In [ ]:
# Initialize figure
fig_1 = matplotlib.pyplot.figure(1)
ax = fig_1.add_subplot(111)
ax.grid(True)

textstr = '\n'.join((
    r'$\mathrm{median}=%.2f$' % (k_eff[0]),
    r'$\sigma=%.2f$ pcm' % (std_k[0])))
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

ax.hist(k_ds_array[:,0], bins='auto', density=True, histtype='stepfilled', color='#0504aa',alpha=0.7)

ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

#ax.legend(loc=0)
ax.set_xlabel(r'$k_{\infty}$')
ax.set_title('Full-core TAP (Serpent): Infinite muliplication factor for BOL')
fig_1.show()

# Initialize figure
fig_2 = matplotlib.pyplot.figure(2)
ax = fig_2.add_subplot(111)
ax.grid(True)
ax.hist(k_ds_array[:,-1], bins='auto', density=True, histtype='stepfilled', color='#0504aa',alpha=0.7)

textstr = '\n'.join((
    r'$\mathrm{median}=%.2f$' % (k_eff[-1]),
    r'$\sigma=%.2f$ pcm' % (std_k[-1])))

ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)

#ax.legend(loc=0)
ax.set_xlabel(r'$k_{\infty}$')

ax.set_title('Full-core TAP (Serpent): Infinite muliplication factor for EOL')
fig_2.show()

In [ ]:
# Initialize figure
fig_keff_std = matplotlib.pyplot.figure(3, figsize=[7,5])
ax = fig_keff_std.add_subplot(111)
ax.grid(True)
ax.plot(days/365.0, std_k, '-s', label='TAP', markerfacecolor="None")

ax.legend(loc=0)
ax.set_ylabel(r'$k_{\infty}$ uncertanty [pcm]')
ax.set_xlabel('EFPY')
ax.set_title('Multiplication factor uncertanty')
fig_keff_std.show()
#fig_keff_std.savefig('keff_std_pwr_vs_tap.png', bbox_inches='tight', dpi=700)

In [ ]:
plot_list = ['U235', 'U236','U238', 'Pu239', 'Xe135', 'I135']
# Initialize figure
fig_mass = matplotlib.pyplot.figure(4, figsize=[8,6])
ax = fig_mass.add_subplot(111)
ax.grid(True)

for iso in plot_list:
    ax.plot(days/365.0, mass[iso], '+-', label=iso, markerfacecolor="None")

ax.legend(loc=0)
ax.set_ylabel(r'Mass of the element in the core [kg]')
ax.set_xlabel('EFPY')
ax.set_title('Inventory of major isotopes in the TAP')
fig_mass.show()
#fig_mass.savefig('keff_std_pwr_vs_tap.png', bbox_inches='tight', dpi=700)

# Initialize figure
fig_rel_error = matplotlib.pyplot.figure(5, figsize=[8,6])
ax = fig_rel_error.add_subplot(111)
ax.grid(True)

for iso in plot_list:
    ax.plot(days/365.0, mass_rel[iso], 'o-', label=iso, markerfacecolor="None")

ax.legend(loc=0)
ax.set_ylabel(r'Mass uncertainty [%]')
ax.set_xlabel('EFPY')
ax.set_title('Uncertainty from Serpent in mass of major isotopes in the TAP')
fig_rel_error.show()
#fig_rel_error.savefig('keff_std_pwr_vs_tap.png', bbox_inches='tight', dpi=700)

In [ ]:
sample_number_list = np.arange(250, len(k_ds_list), 10)
std_samp_conv_k = []
std_samp_conv = []

for num in sample_number_list:
    std_samp_conv_k.append(1e5*np.std(k_ds_array[:num,-1]))
    std_samp_conv.append(100*np.std(depletion[:num,iso_names.index('U235'),-1])/ \
                             np.median(depletion[:num,iso_names.index('U235'),-1]))

# Initialize figure
fig_std_convergance = matplotlib.pyplot.figure(6, figsize=[7,5])
ax = fig_std_convergance.add_subplot(111)
ax.grid(True)
ax.plot(sample_number_list, std_samp_conv, '-s', label='TAP', markerfacecolor="None")

ax.legend(loc=0)
ax.set_ylabel(r'$^{235}$U mass uncertanty [%]')
ax.set_xlabel('Number of samples')
ax.set_title(r'$^{235}$U mass uncertanty convergance')
fig_std_convergance.show()
#fig_std_convergance.savefig('keff_std_pwr_vs_tap.png', bbox_inches='tight', dpi=700)

# Initialize figure
fig_std_convergance_k = matplotlib.pyplot.figure(7, figsize=[7,5])
ax = fig_std_convergance_k.add_subplot(111)
ax.grid(True)
ax.plot(sample_number_list, std_samp_conv_k, '-o', label='TAP', markerfacecolor="None")

ax.legend(loc=0)
ax.set_ylabel(r'$^k_{eff}$ uncertanty [pcm]')
ax.set_xlabel('Number of samples')
ax.set_title(r'$k_{eff}$ uncertanty convergance')
fig_std_convergance_k.show()
#fig_std_convergance.savefig('keff_std_pwr_vs_tap.png', bbox_inches='tight', dpi=700)